In [1]:
import torch

import pandas as pd

from tqdm import tqdm
from torch import nn as nn
from torch import functional as F

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#### Prepare data

In [2]:
path_data = "../dataset/"
buoys_df = pd.read_csv(f"{path_data}prepared_buoy_data.csv", index_col=[0, 1])

kvs_10 = buoys_df.loc['KVS_SvalMIZ_10'].dropna()

In [3]:
X = kvs_10.drop(columns = ['lat', 'lon', 'temp_air', 'temp_surf', 'temp_snow_ice', 'temp_ice', 'sic'])
y = kvs_10['temp_air']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
class BuoyDataset(torch.utils.data.Dataset):
    def __init__(self, features, targets):
        self.X = torch.tensor(features, dtype = torch.float32)
        self.y = torch.tensor(targets.values, dtype = torch.float32).view(-1 , 1)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [6]:
train_dataset = BuoyDataset(X_train, y_train)
test_dataset = BuoyDataset(X_test, y_test)

In [7]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 64, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 64)

#### Define model

In [10]:
class SimpleMLP(nn.Module):
    def __init__(self, in_size = 1, out_size = 1, hidden_size = 32):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(in_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, out_size),
        )

    def forward(self, x):
        return self.network(x)

#### training loop

In [15]:
model = SimpleMLP()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
criterion = nn.MSELoss()
epochs = 1000

prog_bar = tqdm(range(epochs))

for epoch in prog_bar:
    prog_bar.set_description(desc = f"Epoch: {epoch+1}")
    model.train()
    total_loss = 0
    for X, y in train_loader:
        optimizer.zero_grad()
        preds = model(X)

        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    prog_bar.set_postfix({'loss': f"{total_loss:.4f}"})
    

Epoch: 1000: 100%|██████████| 1000/1000 [00:07<00:00, 137.46it/s, loss=14.3841]
